Goal: Usind data about strokes, build a predictor of which conditions are most likely to result in a sroke.

In [1]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
from config import CONFIG
import gradio as gr
import classes.Encoders as enc
import classes.BMICalculator

### EDA

In [2]:
# EDA
data_loca= CONFIG["DATA_FILEPATH"]
child_bmi_data = CONFIG["CHILD_BMI"]
data = pd.read_csv(data_loca)
child_bmi_df = pd.read_csv(child_bmi_data)
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [3]:
len(data)

5110

In [4]:
data_df = data.dropna()
len(data_df)

4909

In [5]:
# round ages to nearest int
data_df["age"] = data_df["age"].round()

/var/folders/18/918s5vr16kl9091zgnswr5gm0000gn/T/ipykernel_25830/4176842324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df["age"] = data_df["age"].round()


In [6]:
# can't calc bmi for ages below 2
data_df = data_df[data_df["age"] > 1]

In [7]:
len(data_df)

4828

In [8]:
# # get histograms of all data to see spread
# import matplotlib.pyplot as plt
# cols = ["gender", "age", "hypertension", "heart_disease","ever_married", "work_type","Residence_type", "avg_glucose_level","bmi","smoking_status","stroke"]
# for col in cols:
#     # counts = data_df[col].value_counts().plot.barh()
#     # fig, ax = plt.subplots()
#     # ax.set_title(f"{col}")
#     # plt.show()

#     categories = data_df[col].value_counts().index
#     counts = data_df[col].value_counts().values
#     plt.barh(categories, counts)
#     plt.title(col)
#     plt.show()


In [9]:
#  look at correlations
# import numpy as np
# import pandas as pd
# from scipy.stats import chi2_contingency

# def cramers_v(x, y):
#     confusion_matrix = pd.crosstab(x, y)
#     chi2 = chi2_contingency(confusion_matrix)[0]
#     n = confusion_matrix.sum().sum()
#     phi2 = chi2 / n
#     r, k = confusion_matrix.shape
#     phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
#     r_corr = r - ((r - 1) ** 2) / (n - 1)
#     k_corr = k - ((k - 1) ** 2) / (n - 1)
#     return np.sqrt(phi2corr / min((k_corr - 1), (r_corr - 1)))

# def plot_categorical_correlation(features):
#     num_features = features.shape[1]
#     correlation_matrix = np.zeros((num_features, num_features))

#     for i in range(num_features):
#         for j in range(num_features):
#             correlation_matrix[i, j] = cramers_v(features[:, i], features[:, j])

#     # Create a heatmap to visualize the correlation matrix
#     fig, ax = plt.subplots(figsize=(8, 8))
#     cax = ax.matshow(correlation_matrix, cmap='coolwarm', vmin=0, vmax=1)
#     plt.colorbar(cax)

#     # Add labels to the heatmap
#     feature_names = [f"Feature {i+1}" for i in range(num_features)]
#     ax.set_xticks(np.arange(len(feature_names)))
#     ax.set_yticks(np.arange(len(feature_names)))
#     ax.set_xticklabels(feature_names)
#     ax.set_yticklabels(feature_names)
#     ax.xaxis.tick_bottom()

#     # Rotate the x-axis labels for better visibility
#     plt.xticks(rotation=45, ha='right')

#     # Display the correlation values in each cell
#     for i in range(num_features):
#         for j in range(num_features):
#             ax.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha='center', va='center', color='black')

#     plt.title("Categorical Correlation Matrix (Cramer's V)")
#     plt.xlabel("Features")
#     plt.ylabel("Features")
#     plt.show()


# plot_categorical_correlation(data_df)


In [10]:

# def plot_correlation_matrix(features):
#     # Calculate the correlation matrix using numpy.corrcoef
#     correlation_matrix = np.corrcoef(features, rowvar=False)

#     # Create a heatmap to visualize the correlation matrix
#     fig, ax = plt.subplots(figsize=(8, 8))
#     cax = ax.matshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
#     plt.colorbar(cax)

#     # Add labels to the heatmap
#     feature_names = [f"Feature {i+1}" for i in range(features.shape[1])]
#     ax.set_xticks(np.arange(len(feature_names)))
#     ax.set_yticks(np.arange(len(feature_names)))
#     ax.set_xticklabels(feature_names)
#     ax.set_yticklabels(feature_names)
#     ax.xaxis.tick_bottom()

#     # Rotate the x-axis labels for better visibility
#     plt.xticks(rotation=45, ha='right')

#     # Display the correlation values in each cell
#     for i in range(len(feature_names)):
#         for j in range(len(feature_names)):
#             ax.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha='center', va='center', color='black')

#     plt.title("Pearson's Correlation Matrix")
#     plt.xlabel("Features")
#     plt.ylabel("Features")
#     plt.show()

# plot_correlation_matrix(data_df)


### BMI conversion data

These files contain the L, M, and S parameters needed to generate exact percentiles and z-scores along with the percentile values for the 3rd, 5th, 10th, 25th, 50th, 75th, 90th, 95th, and 97th percentiles by sex (1=male; 2=female) and single month of age. The smoothed 85th percentile values are included in the BMI-for-age and weight-for-stature tables. Age is listed at the half month point for the entire month; for example, 1.5 months represents 1.0-1.99 months or 1.0 month up to but not including 2.0 months of age. The only exception is birth, which represents the point at birth. To obtain L, M, and S values at finer age or length/stature intervals interpolation could be used.

The LMS parameters are the median (M), the generalized coefficient of variation (S), and the power in the Box-Cox transformation (L). To obtain the value (X) of a given physical measurement at a particular z-score or percentile, use the following equation:

X = M (1 + LSZ)**(1/L), L ≠ 0

Or

X = M exp(SZ), L = 0

where the L, M, and S are the values from the appropriate table corresponding to the age in months of the child (** indicates an exponent, such that M(1+LSZ)**(1/L) means raising (1+LSZ) to the (1/L)th power and then multiplying the M; exp(X) is the exponentiation function, e to the power X). Z is the z-score that corresponds to the percentile. z-scores correspond exactly to percentiles, e.g., z-scores of -1.881, -1.645, -1.282, -0.674, 0, 0.674, 1.036, 1.282, 1.645, and 1.881 correspond to the 3rd, 5th, 10th, 25th, 50th, 75th, 85th, 90th, 95th, and 97th percentiles, respectively.

For example, to obtain the 5th percentile of weight-for-age for a 9-month-old male, we would look up the L, M and S values from the WTAGEINF table, which are L=-0.1600954, M=9.476500305, and S=0.11218624. For the 5th percentile, we would use Z=-1.645. Using the equation above, we calculate that the 5th percentile is 7.90 kg.

To obtain the z-score (Z) and corresponding percentile for a given measurement (X), use the following equation:

((X/M)**L) – 1
Z = ————————-, L≠0
LS

or

Z = ln(X/M)/S ,L=0

where X is the physical measurement (e.g. weight, length, head circumference, stature or calculated BMI value) and L, M and S are the values from the appropriate table corresponding to the age in months of the child (or length/stature). (X/M)**L means raising the quantity (X/M) to the Lth power.

For example, to obtain the weight-for-age z-score of a 9-month-old male who weighs 9.7 kg, we would look up the L, M and S values from the WTAGEINF table, which are L=-0.1600954, M=9.476500305, and S=0.11218624. Using the equation above, we calculate that the z-score for this child is 0.207. This z-score corresponds to the 58th percentile.

Z-scores and corresponding percentiles can be obtained from standard normal distribution tables found in statistics text books. Standard normal tables can also be found on the internet by doing a search on a “standard normal table.” In addition, many computer programs have pre-existing functions that convert Z-scores to percentiles and vice versa.



In [11]:
# chile BMI conversion df
child_bmi_df.head()

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
0,1,24,-2.01118107,16.57502768,0.080592465,14.52095333,14.73731947,15.09032827,15.74164233,16.57502768,17.55718781,18.16219473,18.60948128,19.33801062,19.85985812
1,1,24.5,-1.982373595,16.54777487,0.080127429,14.50347667,14.71929257,15.07117474,15.71962876,16.54777487,17.52129279,18.11954923,18.56110634,19.27889813,19.79194014
2,1,25.5,-1.924100169,16.49442763,0.079233994,14.46882381,14.68360841,15.03335725,15.67634464,16.49442763,17.45135039,18.03668013,18.46729593,19.16465965,19.66102345
3,1,26.5,-1.86549793,16.44259552,0.078389356,14.43459737,14.64843329,14.99619505,15.634035,16.44259552,17.38383656,17.95700228,18.37736191,19.05567423,19.5365754
4,1,27.5,-1.807261899,16.3922434,0.077593501,14.40082828,14.61378626,14.95969047,15.5926798,16.3922434,17.3187102,17.88047101,18.29125345,18.9518675,19.41848805


In [12]:
child_bmi_df.describe()

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
count,439,439,439,439,439,439,439,439,439,439,439,439,439,439,439
unique,3,220,439,439,439,439,439,439,439,439,439,439,439,439,439
top,1,24,-2.01118107,16.57502768,0.080592465,14.52095333,14.73731947,15.09032827,15.74164233,16.57502768,17.55718781,18.16219473,18.60948128,19.33801062,19.85985812
freq,219,2,1,1,1,1,1,1,1,1,1,1,1,1,1


In [14]:
# # drop secondary heading (from joint male and female bmi chart)
# child_bmi_df = child_bmi_df.drop(index = 219)

In [15]:
child_bmi_df["Sex"].unique()

array(['1', '2'], dtype=object)

In [16]:
child_bmi_df.dtypes

Sex       object
Agemos    object
L         object
M         object
S         object
P3        object
P5        object
P10       object
P25       object
P50       object
P75       object
P85       object
P90       object
P95       object
P97       object
dtype: object

In [17]:
child_bmi_df = child_bmi_df.astype(float)
child_bmi_df["Sex"] = child_bmi_df["Sex"].astype(int)

In [18]:
child_bmi_df.dtypes

Sex         int64
Agemos    float64
L         float64
M         float64
S         float64
P3        float64
P5        float64
P10       float64
P25       float64
P50       float64
P75       float64
P85       float64
P90       float64
P95       float64
P97       float64
dtype: object

In [19]:
child_bmi_df

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
0,1,24.0,-2.011181,16.575028,0.080592,14.520953,14.737319,15.090328,15.741642,16.575028,17.557188,18.162195,18.609481,19.338011,19.859858
1,1,24.5,-1.982374,16.547775,0.080127,14.503477,14.719293,15.071175,15.719629,16.547775,17.521293,18.119549,18.561106,19.278898,19.791940
2,1,25.5,-1.924100,16.494428,0.079234,14.468824,14.683608,15.033357,15.676345,16.494428,17.451350,18.036680,18.467296,19.164660,19.661023
3,1,26.5,-1.865498,16.442596,0.078389,14.434597,14.648433,14.996195,15.634035,16.442596,17.383837,17.957002,18.377362,19.055674,19.536575
4,1,27.5,-1.807262,16.392243,0.077594,14.400828,14.613786,14.959690,15.592680,16.392243,17.318710,17.880471,18.291253,18.951867,19.418488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,2,237.5,-2.353501,21.689489,0.151755,17.435152,17.822557,18.480142,19.788872,21.689489,24.384258,26.397231,28.130338,31.596640,34.818095
435,2,238.5,-2.350529,21.701083,0.152221,17.434275,17.822591,18.481819,19.794202,21.701083,24.406856,26.429839,28.172905,31.663237,34.912504
436,2,239.5,-2.346962,21.711892,0.152716,17.431988,17.821267,18.482230,19.798457,21.711892,24.429104,26.462429,28.215745,31.730686,35.008306
437,2,240.0,-2.344958,21.716999,0.152975,17.430311,17.820090,18.481956,19.800176,21.716999,24.440096,26.478720,28.237271,31.764743,35.056751


In [20]:
# child_bmi_df

t_age = 168.5
t_gender = 2
t_age_mask = child_bmi_df["Agemos"] == t_age
t_gender_mask = child_bmi_df["Sex"] == t_gender
            
t_lookup_row = child_bmi_df[t_age_mask & t_gender_mask]
t_lookup_row

,Sex,Agemos,L,M,S,P3,P5,P10,P25,P50,P75,P85,P90,P95,P97
365,2,168.5,-1.977074,19.352572,0.151256,15.442351,15.807529,16.423776,17.635089,19.352572,21.688194,23.34689,24.711407,27.255974,29.396856


## Encode data for training

In [21]:
# split data by age 
child_df = data_df[data_df["age"] <= 20]
adult_df = data_df[data_df["age"] > 20]

In [22]:
# encode bmi for each df
enc_child_bmi =  enc.ChildBMIEncoder(bmi_table= child_bmi_df, data_table= child_df).encode()
enc_adult_bmi = enc.AdultBMIEncoder(adult_df).encode()

/Users/jadeevans/Documents/Code/machine-learning/biomed-datasci-projs/stroke-prediction/classes/Encoders.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data_table['bmi_buckets'] = bmi_buckets


In [26]:
new_df = pd.concat([enc_child_bmi,enc_adult_bmi], axis = 1)
new_df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,...,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
245,49669.0,Female,14.0,0.0,0.0,No,children,Rural,57.93,30.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,30669.0,Male,3.0,0.0,0.0,No,children,Rural,95.12,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,16523.0,Female,8.0,0.0,0.0,No,Private,Urban,110.89,17.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,46136.0,Male,14.0,0.0,0.0,No,Never_worked,Rural,161.28,19.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,19584.0,Female,20.0,0.0,0.0,No,Private,Urban,84.62,19.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,57.0,0.0,0.0,Yes,Private,Rural,77.93,21.7,never smoked,0.0
5106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,81.0,0.0,0.0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0.0
5107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0,0.0,0.0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0.0
5108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51.0,0.0,0.0,Yes,Private,Rural,166.29,25.6,formerly smoked,0.0


In [ ]:
# bmi_cat = calc_bmi_percentile(categories, child_bmi_df)
new_df = de.get_bmi_bucket(data_table = data_df, bmi_table =child_bmi_df )
new_df



In [ ]:
age_obj = enc.AgeEncoder(new_df,"age")
encode_age = age_obj.encode()
# new_df = encode_age.df()
# new_df
encode_age


In [ ]:
de.encode_bmi(new_df,"bmi_buckets")

new_df

In [ ]:
de.encode_gender(new_df)
de.encode_marriage(new_df)
de.encode_residence(new_df)
de.encode_smoke_stat(new_df,"smoking_status")
new_df

In [ ]:
new_df["stroke"]

In [ ]:
def get_train_df(df, drop_cols, label_col):
    """
    Input: 
        df with encoded training data
        [drop_cols]; columns names to drop from df
        "label_col"; name of label column
    Output: 
        training data df, label df
    """
    labels = df[label_col]
    df.drop(label_col, axis = 1)
    train_df = df.drop(drop_cols, axis = 1)

    return train_df, labels

In [ ]:
# get train df, remove unecceasry columns
# removed avg glucose level for now (for demo)
# drop_cols = ["id", "gender", "age", "ever_married", "work_type", "Residence_type", "bmi","smoking_status", "bmi_buckets","avg_glucose_level"]
drop_cols = ["id", "gender", "ever_married", "work_type", "Residence_type", "bmi","smoking_status", "bmi_buckets","avg_glucose_level"]
train_df, labels = get_train_df(new_df, drop_cols, "stroke")
train_df

In [ ]:
train_df.columns

In [ ]:
labels

In [ ]:
# split into testing and training data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df, labels, test_size = 0.2, random_state = 42)

In [ ]:
X_train.dtypes

In [ ]:
# use SVM for classification
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(X_train, y_train)

In [ ]:
predictions = classifier.predict(X_test)

In [ ]:
classifier.score(X_test,y_test)

In [ ]:
classifier.get_params()

In [ ]:
classifier.predict_proba(X_test)

In [ ]:
# more metrics
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

In [ ]:

classification_report(y_test,predictions)

Try to rerain using k-folds

In [ ]:
# new test-train splits
X_train2, X_test2, y_train2, y_test2 = train_test_split(train_df, labels, test_size = 0.3, random_state = 12)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

model = classifier

#use k-fold CV to evaluate model
scores = cross_val_score(model, X_train2, y_train2, scoring=make_scorer(accuracy_score),
                         cv=cv, n_jobs=-1)

scores

In [ ]:
c2 = classifier
c2.fit(X_train2,y_train2)
pred2 = c2.predict(X_test2)
c2.score(X_test2,y_test2)



## DEMO

In [ ]:
# demo support functions

def request_age(age):
    return age

def encode_age(df):
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    categories = pd.cut(df.Age, bins, labels=False)
    df.Age = categories
    return df

def request_sex(sex):
    return "Male" if sex.lower() == "male" else "Female"

def hypertension(text):
    feature = 1 if text.lower() == "yes" else 0
    return feature

def heart_disease(text):
    feature = 1 if text.lower() == "yes" else 0
    return feature

    

def bmi_standard(weight_lb,height_ft_in, age):
    """ 
    Inputs:
        height_ft_in = (height feet, height in)
        weight_lb = 
    """
    height = (height_ft_in[0] * 12) + height_ft_in[1]
    bmi = weight_lb / (height **2)

    return bmi

    

def bmi_imperial(weight_kg, height_m,age):

    bmi = weight_kg / (height_m **2)

    return bmi

def req_marriage(text):
    if text.lower() == "yes":
        feature = "Yes"
    else: 
        feature = "No"
    return feature

def red_residence(text):
    if text.lower()== 

In [ ]:
def bmi_standard(weight_lb,height_ft_in, age):
    """ 
    Inputs:
        height_ft_in = (height feet, height in)
        weight_lb = 
    """
    height = (height_ft_in[0] * 12) + height_ft_in[1]
    bmi = weight_lb / (height **2)

    return bmi

    

def bmi_imperial(weight_kg, height_m,age):

    bmi = weight_kg / (height_m **2)

    return bmi

In [ ]:
# dropdown data
gr.CheckboxGroup(["Male", "Female"], label="Gender?")
gr.Dropdown(label = "Residence Type",choices =["Urban", "Rural"],show_label = True, interactive = True)

gr.Dropdown(label = "Smoking Status",choices = ["smokes","never smoked", "formerly smoked", "Unknown"],show_label = True)

gr.CheckboxGroup(["Urban", "Rural"], label="Residence Type")
# convert to 1 or 0 
gr.CheckboxGroup(["Yes", "No"], label="Hypertension?")
gr.CheckboxGroup(["Yes", "No"], label="Heart Disease?")

In [ ]:
# get units of height and weight
# get standard height as (feet,inches)

'age', 'hypertension', 'heart_disease', 'stroke', 'bmi_under',
       'bmi_healthy', 'bmi_over', 'bmi_obese', 'is_male', 'c_ever_married',
       'live_urban', 'ss_1', 'ss_2', 'ss_3', 'ss_4'
 

In [ ]:
# make an encoder interface (assembly line)

In [ ]:
# make enocde classes (for each function)


In [ ]:
# create a predict dict class and initialize with the object

In [ ]:
# example:


prediction_dict = {
                    "height": None, 
                    "height_unit": None, 
                    "weight": None, 
                    "weight_unit": None, 
                    "gender": None, 
                    "age": None, 
                    "marriage_stat": None, 
                    "residence": None, 
                    "smoke_stat": None, 
                    "hypertension": None,
                    "heart_disease": None
                }

def predict_survival(prediction_dict):
    """ 
    Input: 
        {prediction_dict};
            "height": None, 
            "height_unit": None, 
            "weight": None, 
            "weight_unit": None, 
            "gender": None, 
            "age": None, 
            "marriage_stat": None, 
            "residence": None, 
            "smoke_stat": None, 
            "hypertension": None,
            "heart_disease": None
    Output:
        Preddicton of stroke (Yes or No)
    """

  prediction_dict["height"] = 
        ["height_unit"]
        ["weight"]
        "weight_unit"
        "gender"
        "age"
        "marriage_stat"
        "residence"
        "smoke_stat"
        "hypertension"
        "heart_disease"
    if height_unit == "kg" and weight_unit == "m":
        bmi = bmi_imperial(weight_kg= weight, height_m= height, age= age)
    else:
        bmi = bmi_standard(weight_lb= weight, height_ft_in=height, age = age)
    
    # initialize df
    df = pd.DataFrame.from_dict(
        {
            "age": 0,
            "gender": gender
            "hypertension": 0,
            "heart_disease": 0,
            "bmi" = bmi
            "live_urban": 0,
            "ever_married": marriage_stat,
            "Residence_type": residence,
            "smoking_status": smoke_stat
                    }
    )
    df = de.get_bmi_bucket(data_table = data_df, bmi_table =child_bmi_df )
    df = de.encode_bmi(df,"bmi_buckets")
    # df = de.encode_ages(df,"age")
    df = de.encode_gender (df)
    df = de.encode_marriage(df)
    df = de.encode_residence(df)
    df = de.encode_smoke_stat(df,"smoking_status")

    drop_cols = ["gender", "bmi"]

    pred = classifier.predict(df)
    return {"Stroke likely": float(pred[0]), "Survives": float(pred[1])}


demo = gr.Interface(
    predict_survival,
    [
        gr.Dropdown(["first", "second", "third"], type="index"),
        "checkbox",
        gr.Slider(0, 80, value=25),
        gr.CheckboxGroup(["Sibling", "Child"], label="Travelling with (select all)"),
        gr.Number(value=20),
        gr.Radio(["S", "C", "Q"], type="index"),
    ],
    "label",
    examples=[
        ["first", True, 30, [], 50, "S"],
        ["second", False, 40, ["Sibling", "Child"], 10, "Q"],
        ["third", True, 30, ["Child"], 20, "S"],
    ],
    interpretation="default",
    live=True,
)

if __name__ == "__main__":
    demo.launch()

In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"



# def predict_stroke(data):
#     """ 
#     hypertension (y/n)
# heart disease (y/n)
# avg glucose level (?)
# bmi (sub for weight and calculate?)
# ever married (y/n/child)
# age 
# neighborhood (urban, rural)
# gender
# smoking status (former, never, yes,  )
# """
#     if passenger_class is None or embark_point is None:
#         return None
#     df = pd.DataFrame.from_dict(
        



#     )
    
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch()